# LLMS

## Set up your `TOGETHER_API_KEY`

In [ ]:
import os

if os.environ.get("TOGETHER_API_KEY") is None:
   os.environ["TOGETHER_API_KEY"] = "paste_your_api_key_here"

## Install Packages to Call the LLM

For this part we set up the LLM.

In [ ]:
!pip3 install openai

In [ ]:
import openai
import textwrap

def wrap_text(text, max_width=80):
    """
    Wraps the text to the specified max_width, preserving line breaks and formatting.
    """
    lines = text.splitlines()  # Split the text into lines
    wrapped_lines = []

    for line in lines:
        if line.strip():  # Skip empty lines
            wrapped_line = textwrap.fill(line, max_width, initial_indent='', subsequent_indent='')
            wrapped_lines.extend(wrapped_line.splitlines())  # Preserve line breaks
        else:
            wrapped_lines.append('')  # Keep empty lines

    return '\n'.join(wrapped_lines)


def print_prompt_and_response(prompt, response):
    print("Prompt: ")
    print(prompt)
    print("")
    print("Response: ")
    print(response)

def get_completion(prompt, temperature=0.0):
    client = openai.OpenAI(api_key=os.environ.get("TOGETHER_API_KEY"),
                           base_url="https://api.together.xyz/v1")

    chat_completion = client.chat.completions.create(
                           model="NousResearch/Nous-Hermes-2-Mixtral-8x7B-SFT",
                           #model="zero-one-ai/Yi-34B-Chat",
                           messages=[{"role": "user", "content": prompt}],
                           temperature=temperature,
                           max_tokens=1024)

    response = chat_completion.choices[0].message.content
    response = response.lstrip()
    wrapped_response = wrap_text(response)
    return wrapped_response

## Calling an LLM with `get_completion`

First set up a prompt we can use:

In [ ]:
prompt = "Why is the sky blue? In 50 words or less."

Now we call the`get_completion` function. This function takes our prompt
as argument and sends it off to the LLM. The LLM returns a response.
This function returns that response.

In [ ]:
response = get_completion(prompt)
print(response)

Sometimes its nice to print out the prompt and the response when we are
working like this. We can use the function `print_prompt_and_response`
and pass it the prompt and the response we got from the `get_completion`
function.

In [ ]:
response = get_completion(prompt)
print_prompt_and_response(prompt, response)

## Calling an LLM with the `temperature`

Temperature is one of the settings that most LLMs have. You can set the
temperature from 0.0 up to some maximum (like 1.0 or 1.2). Sometimes you
can go higher.

Roughly

1.  Low temperature (eg 0.2-0.5)
    -   Generates more predictable and conservative text
2.  Medium temperature (eg. 0.6-0.8)
    -   Balances between creativity and coherence
3.  High temperature (eg. 0.9-1.2)
    -   Generates more creative diverse, and unpredictable text

First lets set up a prompt to call several times with different
temperatures.

In [ ]:
prompt = "Why is the sky blue? In 50 words or less"

### Calling with `temperature=0.0`

`temperature` is parameter you can pass to the `get_completion`
function. You see this in the code below.

Here we call the LLM 3 times with temperature 0.0

In [ ]:
response = get_completion(prompt, temperature = 0.0)
print(response)
print("------")

response = get_completion(prompt, temperature = 0.0)
print(response)
print("------")

response = get_completion(prompt, temperature = 0.0)
print(response)
print("------")

### Calling with `temperature=0.4`

Here we call the LLM 3 times with temperature 0.4

In [ ]:
response = get_completion(prompt, temperature = 0.4)
print(response)
print("------")

response = get_completion(prompt, temperature = 0.4)
print(response)
print("------")

response = get_completion(prompt, temperature = 0.4)
print(response)
print("------")

### Calling with `temperature=1.0`

In [ ]:
response = get_completion(prompt, temperature = 1.0)
print(response)
print("------")

response = get_completion(prompt, temperature = 1.0)
print(response)
print("------")

response = get_completion(prompt, temperature = 1.0)
print(response)
print("------")

## Calling the LLM using a template

Remember the template for having the LLM give us the name of a company
that makes a particular thing?

In [ ]:
template = "Recommend a name for a company that makes {what}"

Lets set that up and call it with `what="cars"`:

In [ ]:
prompt=template.format(what="cars")
response = get_completion(prompt)
print_prompt_and_response(prompt, response)

## Change the template variable `{what}`

Let’s call the LLM using `what="books"` as the template variable.

In [ ]:
prompt=template.format(what="books")
response = get_completion(prompt)
print_prompt_and_response(prompt, response)

## Add A Template Variable `{num}`

In [ ]:
template = "Recommend {num} names for a company that makes {what}"

Lets call it with `num=3` and `what="cars"`:

In [ ]:
prompt=template.format(num=3, what="cars")
response = get_completion(prompt)
print_prompt_and_response(prompt, response)

### Add a Template Variable `{format}`

Lets see if we can introduce some formatting to the output:

In [ ]:
template ="""Recommend {num} names for a company that makes {what}.
Give your answer in {format}"""

Call the LLM:

In [ ]:
prompt=template.format(num=3, what="cars", format="A bulleted list using upper case")
response = get_completion(prompt)
print_prompt_and_response(prompt, response)

### Using a template to analyze reviews

Finally in this example, we use a template for asking the LLM to analyze
some reviews of a product.

This shows how the template idea works. We can work on the template
separately from the reviews in just one place, that way once the
template works we can apply it to any number of reviews.

Here are the reviews:

In [ ]:
first_review = """So close! Aside from the technical issues — I ordered a small
of the less busy face pattern, and received a medium in the more busy face
pattern option instead. I’m 5’6” 135-140lbs. The sizes were European. This
medium was a little big but still would have felt cute, if not for the collar
and neckline. The collar has a high backing that doesn’t allow it to fall too
well around your neck, and since the fabric is very thin and one-sided, you can
see the back of the pattern when the collar is open (see photos). While okay
for casual wear, and has a great boxy cut, the exposed back of fabric feels a
bit cheap. Shirt is a good length for tucking in and a bit long otherwise."""

second_review = """I love the pattern on this shirt, and I wanted to keep it so
badly!! A few issues, though, made me send it back. 1 - The shirt looks much
shinier/glossier on the models, and it's not. It's not shiny at all. It's a
very nice fabric that is lightweight, but it hangs a lot differently that what
was shown. 2 - The front is very short. The back hangs down nicely, but if you
want to hide any tummy issues, this shirt won't do it for you. I also felt like
my arms couldn't come up any or the shirt would be too high. I do a lot of
presenting so this was a no go for me. 3 - The stitching on the front pulls
oddly, so although it was big enough to button across the chest area, the
stitching made it seem as though it was pulling even when it wasn't. Could just
be my experience, but I don't like looking like the shirt is barely holding on,
ha. I am 5'4 with a more muscular build (wide shoulders too) so that might be
part of it. I love this shirt and wish it would work for me, but the shortness
of it and the way it pulls made it a no-go."""

And here is our template.

In [ ]:
# Here is our template
template="""
You will be given a review in <text> tags below.

- Summarize the review using around {length} words
- On a separate line give {num} keywords that identify the main concerns in the review

<text>
{review}
</text>
"""

Lets pass in the first review.

In [ ]:
prompt=template.format(review=second_review, num=3, length=100)
response = get_completion(prompt)
print_prompt_and_response(prompt, response)

Lets pass in the second review.

In [ ]:
prompt=template.format(review=second_review, num=5, length=50)
response = get_completion(prompt)
print_prompt_and_response(prompt, response)